In [18]:
import numpy as np
from skimage.morphology import skeletonize, medial_axis, watershed, disk
from skimage import data
from scipy import ndimage as ndi
from scipy.spatial import distance
import matplotlib.pyplot as plt
from skimage.util import invert
from skimage.io import imshow, imread
from skimage.color import rgb2gray
from skimage.feature import peak_local_max, canny
from scipy import ndimage
from skimage.filters import sobel
from skimage.measure import label, regionprops
from skimage.filters import rank

%matplotlib inline
import numpy as np
from skimage import io
from skimage.feature import canny
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.morphology import watershed, disk
from skimage.filters import rank
from skimage.util import img_as_ubyte
from skimage import segmentation
np.set_printoptions(threshold=np.nan)
import math, skimage
from skimage import morphology
from skimage.measure import label, regionprops
from skimage.feature import corner_harris, corner_subpix, corner_peaks

def centroid(image):
    #Uses watershed to segment each wing into separate cells
    img = image
    denoised = rank.median(img, disk(2))
    markers = rank.gradient(denoised, disk(5)) < 10
    markers = ndi.label(markers)[0]
    gradient = rank.gradient(denoised, disk(2))
    labels = watershed(gradient, markers)
    cleared = segmentation.clear_border(labels)
    denoised = rank.median(img, disk(2))
    gradient = rank.gradient(denoised, disk(5))
    thres = 30
    gradient[gradient > thres] = 255

    labeled_wing = cleared
    regions = regionprops(labeled_wing)

    #Filters regions based on area
    filtered_regions = []
    filtered_regions_area = []
    for props in regions:
        area = props.area
        if area>1000 and area<400000:
            filtered_regions.append(props)
            filtered_regions_area.append(area)

    #Plots centroids of regions
    #for region in filtered_regions:
    #    y0, x0 = region.centroid
    #    plt.scatter([x0], [y0])
    #plt.title("Filtered Regions, Plotted Centroids")
    #plt.show()
    #Finds the centroid of all the region centers
    # sum_x, sum_y = 0, 0
    centroids = []
    centroids_x = []
    centroids_y = []
    for region in filtered_regions:
        y0, x0 = region.centroid
        region_centroid = (x0, y0)
        centroids.append(region_centroid)
        centroids_x.append(x0)
        centroids_y.append(y0)
        
    avg_x = np.mean(centroids_x)
    avg_y = np.mean(centroids_y)
    
    #Calculates the distance between each centroid to the center
    distances_from_center = []
    for centroid in centroids:
        distances_from_center.append(distance.euclidean(centroid, (avg_x, avg_y)))

    #Normalizes distances from center
    distance_sd = np.std(distances_from_center)
    distance_mean = np.mean(distances_from_center)
    dist_from_cent_norm = [dist_from_center/distance_mean for dist_from_center in distances_from_center]

    #Normalizes areas 
    area_sd = np.std(filtered_regions_area)
    area_mean = np.mean(filtered_regions_area)
    area_norm = [area/area_mean for area in filtered_regions_area ]    
    
    #Weights each region centroid based on distance to the center, and size
    distance_weight = 1
    size_weight = .5

    weighted_values = []
    for i in range(len(filtered_regions)):
        weighted_values.append(size_weight*area_norm[i] - distance_weight*dist_from_cent_norm[i])

    max_index = weighted_values.index(max(weighted_values))
    center_cell = filtered_regions[max_index]
    center_cell_x, center_cell_y = center_cell.centroid[1], center_cell.centroid[0]
    
    implot = plt.imshow(labeled_wing)
    plt.scatter([center_cell_x], [center_cell_y])
    return center_cell.centroid[0], center_cell.centroid[1]

# centroid("enhanced_image/060 Lasioglossum nymphaerum f left 4x.jpg")
#print("Centroid", centroid("enhanced_image/060 Lasioglossum nymphaerum f left 4x.jpg"))

In [19]:
from ipywidgets import interact, fixed

image = imread("../data/enhanced_image/060 Lasioglossum nymphaerum f left 4x.jpg")[..., 1]

@interact(deg=(0, 360, 1))
def center_cell_after_rotation_by(deg=0):
    img = skimage.transform.rotate(image, deg, resize=True)
    C = centroid(img)
    
    plt.imshow(img, cmap='gray')
    plt.scatter([C[1]], [C[0]])
    plt.show()

interactive(children=(IntSlider(value=0, description='deg', max=360), Output()), _dom_classes=('widget-interact',))

In [23]:
# from batch_runner import BatchRunner

# brunner = BatchRunner("../data/enhanced_image")
# brunner.runBatch(centroid)